In [90]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviakompaniya/sample_submission.csv
/kaggle/input/aviakompaniya/train_dataset.csv
/kaggle/input/aviakompaniya/test_dataset.csv


In [91]:
from sklearn import metrics
from sklearn import tree
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
#from  sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
#from sklearn.ensemble import RandomForestClassifier

In [92]:
d = [pd.read_csv("/kaggle/input/aviakompaniya/" +x+ "_dataset.csv") for x in ['train','test']]

In [93]:
mean = d[0]['Arrival Delay in Minutes'].sum()/len(d[0]['Arrival Delay in Minutes'])
d[0]['Arrival Delay in Minutes']=d[0]['Arrival Delay in Minutes'].fillna(mean)
mean = d[1]['Arrival Delay in Minutes'].sum()/len(d[1]['Arrival Delay in Minutes'])
d[1]['Arrival Delay in Minutes']=d[1]['Arrival Delay in Minutes'].fillna(mean)

In [94]:
print(d[0].shape)
print(d[1].shape)

(10000, 24)
(4000, 23)


In [95]:
d[0].drop('id',axis=1,inplace=True)
d[1].drop('id',axis=1,inplace=True)

In [96]:
list(d[0].columns)

['Gender',
 'Customer Type',
 'Age',
 'Type of Travel',
 'Class',
 'Flight Distance',
 'Inflight wifi service',
 'Departure/Arrival time convenient',
 'Ease of Online booking',
 'Gate location',
 'Food and drink',
 'Online boarding',
 'Seat comfort',
 'Inflight entertainment',
 'On-board service',
 'Leg room service',
 'Baggage handling',
 'Checkin service',
 'Inflight service',
 'Cleanliness',
 'Departure Delay in Minutes',
 'Arrival Delay in Minutes',
 'satisfaction']

In [97]:
d_n = ['Age',
       'Flight Distance',
       'Inflight wifi service',
       'Departure/Arrival time convenient',
       'Ease of Online booking','Gate location',
       'Seat comfort','Inflight entertainment',
       'On-board service',
       'Leg room service',
      'Baggage handling',
      'Checkin service',
      'Inflight service',
      'Cleanliness',
      'Departure Delay in Minutes',
     'Arrival Delay in Minutes']
d_s = ['Gender',
       'Customer Type',
       'Type of Travel',
       'Class']

In [98]:
full_pyp = ColumnTransformer([
    ('num', StandardScaler(), d_n),
    ('st_r', OneHotEncoder(), d_s)
])

In [99]:
y = d[0]['satisfaction'].copy()
x = d[0].drop('satisfaction',axis=1)


In [100]:
x_train = full_pyp.fit_transform(x)
x_test = full_pyp.fit_transform(d[1])

In [101]:
xgb = XGBClassifier()
xgb.fit(x_train,y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [102]:
xgb_pred = xgb.predict(x_test)

In [103]:
df_s['satisfaction'] = xgb_pred
df_s

,id,satisfaction
0,1,0
1,2,1
2,3,1
3,4,1
4,5,0
...,...,...
3995,3996,1
3996,3997,0
3997,3998,0
3998,3999,1
